# imports..

In [2]:
from research.research_engine import build_ResearchEngine
from strategies.vault_betsizing import YieldStrategy
from strategies.vault_backtest import VaultBacktestEngine
import json
import yaml
import numpy as np
import pandas as pd
from copy import deepcopy

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# load data

In [4]:
vault = 'DynYieldE'
metadata = pd.read_csv(f"C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/{vault}_pool_metadata.csv")
with open(f"C:/Users/david/singularity/midfreq/config/{vault.lower()}.yaml", 'r') as file:
    parameters= yaml.safe_load(file)
pool_map = metadata.loc[metadata['pool'].isin(parameters['input_data']['selected_instruments']),['pool','project','symbol']].set_index('pool').T.to_dict()
pool_map = {key: '{}_{}'.format(x['project'], x['symbol']) for key, x in pool_map.items()}
index_map = {f'weight_{i}': f'w_{value}' for i, value in enumerate(pool_map.values())}
index_map |= {f'yield_{i}': f'y_{value}' for i, value in enumerate(pool_map.values())} 

### naive optimization - 1bp swap_cost

In [13]:
parameter_grid = {"cap": 9,
                  "halflife": "1s",
                  "cost": 0.0001,
                  "gaz": False,
                  "assumed_holding_days": 9999,
                  "base_buffer": 0.15,
                  "concentration_limit": 1.0}
filename = "C:/Users/david/singularity/midfreq/logs/{}/{}_{}_{}_{}_{}_{}_{}__backtest.csv".format(vault.lower(), *parameter_grid.values())
backtest = pd.read_csv(filename, header=[0,1])

In [14]:
backtest['pnl']["tx_cost"].iloc[0] = 0.001
cum_tx_cost = backtest['pnl']["tx_cost"].cumsum()
apy = (backtest['pnl']['wealth'] / backtest['pnl']['wealth'].shift(1) -1)*365/1*100
max_apy = backtest['full_apy'].max(axis=1)*100
max_apy.name = 'max_apy'

subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.line(apy,render_mode="webgl",)
fig3 = px.line(max_apy, render_mode="webgl")
fig3.update_traces(line={"dash": 'dot'})
fig2 = px.line(cum_tx_cost, render_mode="webgl",)
fig2.update_traces(yaxis="y2")
subfig.add_traces(fig.data + fig2.data + fig3.data)
subfig.layout.xaxis.title = "time"
subfig.layout.yaxis.title = "apy"
subfig.layout.yaxis.range = [0, 15]
subfig.layout.yaxis2.title = "tx_cost"

# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.show()

print(filename)
apy.mean()

C:/Users/david/singularity/midfreq/logs/dynyielde/9_1s_0.0001_False_9999_0.15_1.0__backtest.csv


5.408944953488474

### naive optimization - 10bps swap_cost

In [7]:
parameter_grid = {"cap": 9,
                  "halflife": "1s",
                  "cost": 0.001,
                  "gaz": False,
                  "assumed_holding_days": 9999,
                  "base_buffer": 0.15,
                  "concentration_limit": 1.0}
filename = "C:/Users/david/singularity/midfreq/logs/{}/{}_{}_{}_{}_{}_{}_{}__backtest.csv".format(vault.lower(), *parameter_grid.values())
backtest = pd.read_csv(filename, header=[0,1])

In [8]:
backtest['pnl']["tx_cost"].iloc[0] = 0.001
cum_tx_cost = backtest['pnl']["tx_cost"].cumsum()
apy = (backtest['pnl']['wealth'] / backtest['pnl']['wealth'].shift(1) -1)*365/1*100
max_apy = backtest['full_apy'].max(axis=1)*100
max_apy.name = 'max_apy'

subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.line(apy,render_mode="webgl",)
fig3 = px.line(max_apy, render_mode="webgl")
fig3.update_traces(line={"dash": 'dot'})
fig2 = px.line(cum_tx_cost, render_mode="webgl",)
fig2.update_traces(yaxis="y2")
subfig.add_traces(fig.data + fig2.data + fig3.data)
subfig.layout.xaxis.title = "time"
subfig.layout.yaxis.title = "apy"
subfig.layout.yaxis.range = [0, 15]
subfig.layout.yaxis2.title = "tx_cost"

# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.show()

print(filename)
apy.mean()

C:/Users/david/singularity/midfreq/logs/dynyielde/9_1s_0.001_False_9999_0.15_1.0__backtest.csv


-26.177151840540485

# naive optimization - 0.4 concentration_limit

In [9]:
parameter_grid = {"cap": 9,
                  "halflife": "1s",
                  "cost": 0.001,
                  "gaz": False,
                  "assumed_holding_days": 9999,
                  "base_buffer": 0.15,
                  "concentration_limit": 0.4}
filename = "C:/Users/david/singularity/midfreq/logs/{}/{}_{}_{}_{}_{}_{}_{}__backtest.csv".format(vault.lower(), *parameter_grid.values())
backtest = pd.read_csv(filename, header=[0,1])

In [10]:
backtest['pnl']["tx_cost"].iloc[0] = 0.001
cum_tx_cost = backtest['pnl']["tx_cost"].cumsum()
apy = (backtest['pnl']['wealth'] / backtest['pnl']['wealth'].shift(1) -1)*365/1*100
max_apy = backtest['full_apy'].max(axis=1)*100
max_apy.name = 'max_apy'

subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.line(apy,render_mode="webgl",)
fig3 = px.line(max_apy, render_mode="webgl")
fig3.update_traces(line={"dash": 'dot'})
fig2 = px.line(cum_tx_cost, render_mode="webgl",)
fig2.update_traces(yaxis="y2")
subfig.add_traces(fig.data + fig2.data + fig3.data)
subfig.layout.xaxis.title = "time"
subfig.layout.yaxis.title = "apy"
subfig.layout.yaxis.range = [0, 15]
subfig.layout.yaxis2.title = "tx_cost"

# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.show()

print(filename)
apy.mean()

C:/Users/david/singularity/midfreq/logs/dynyielde/9_1s_0.001_False_9999_0.15_0.4__backtest.csv


-17.226152098307757

# heatmaps

In [11]:
parameter_grid = {"cap": [0.2],
                  "halflife": ["1s", "3d", "10d", "30d"],
                  "cost": [0.001],
                  "gaz": [False],
                  "assumed_holding_days": [3, 8, 13, 18, 23, 28, 32, 9999],
                  "base_buffer": [0.15],
                  "concentration_limit": [0.4,
                                          0.7, 1.0]}

In [15]:
grid

,cap,halflife,swap_cost,gaz,assumed_holding_days,base_buffer,concentration_limit,perf,tx_cost,avg_entropy
0,0.2,1s,0.0001,False,3,0.15,0.4,"Solver 'ECOS' failed. Try another solver, or s...",0.001,"Solver 'ECOS' failed. Try another solver, or s..."
1,0.2,1s,0.0001,False,3,0.15,0.7,"Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s..."
2,0.2,1s,0.0001,False,3,0.15,1.0,"Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s..."
3,0.2,1s,0.0001,False,8,0.15,0.4,0.04840175522539208,0.016399316078180834,0.40417694189433667
4,0.2,1s,0.0001,False,8,0.15,0.7,"Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s..."
...,...,...,...,...,...,...,...,...,...,...
763,9.0,30d,0.0050,False,32,0.15,0.7,0.03150087770464085,0.008640431480383556,0.247010147781055
764,9.0,30d,0.0050,False,32,0.15,1.0,0.03351976279532568,0.008642623691895933,0.1306433718667029
765,9.0,30d,0.0050,False,9999,0.15,0.4,"Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s..."
766,9.0,30d,0.0050,False,9999,0.15,0.7,"Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s...","Solver 'ECOS' failed. Try another solver, or s..."


In [12]:
grid = pd.read_csv(f"C:/Users/david/singularity/midfreq/logs/{vault.lower()}/grid.csv", index_col=0)
grid['tx_cost'].iloc[0] = 0.001
grid = grid.rename(columns={'cost': 'swap_cost'})
all_index = {col: grid[col].unique() for col in grid.columns}
print(grid.columns)

grid['churn in d'] = grid.apply(lambda x: 2 * 365 * x['swap_cost']/x['tx_cost']/100, axis=1)

format_dict = {key: (lambda x: x) for key in grid.columns}
format_dict['perf'] = lambda x: f'{x:.1%}'
format_dict['tx_cost'] = lambda x : f'{x:.0f}'
format_dict['avg_entropy'] = lambda x: f'{x:.0%}'

def display_heatmap(metrics, ind, col, filtering):
    fig = plt.figure(figsize = (20,20)) # width x height
    
    # filter
    filtered_grid = grid[np.logical_and.reduce([
            grid[filter_c].isin([filter_v])
        for filter_c, filter_v in filtering.items()])]
    
    # pivot and display
    for i, values in enumerate(metrics):
        for j, column in enumerate(col):
            df = grid.pivot_table(values=values, index=ind, columns=col)*100
            ax = fig.add_subplot(len(col), len(metrics), i+j+1)
            ax.set_title(f'{values} by {column}')
            sns.heatmap(data=df, ax=ax, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12})

Index(['cap', 'halflife', 'swap_cost', 'gaz', 'assumed_holding_days',
       'base_buffer', 'concentration_limit', 'perf', 'tx_cost', 'avg_entropy'],
      dtype='object')


TypeError: unsupported operand type(s) for /: 'float' and 'str'

## churn vs     activeness / predictor halflife -> halflife = 3d, horizon = 30

In [ ]:
metrics = ['perf', 'churn in d']
ind = ['assumed_holding_days']
col = ['halflife']
filtering = {
    'cap': 0.2,
#    'halflife': "30d",
    'swap_cost': 1e-3,
    'gaz': False,
#    'assumed_holding_days': 30,
    'base_buffer': 0.15,
    'concentration_limit': 0.7,
}
display_heatmap(metrics, ind, col, filtering)

## entropy vs     concentration x activeness

In [ ]:
filtering = {
    'cap': 0.2,
    'halflife': '30d',
    'swap_cost': 5e-4,
    'gaz': False,
    'assumed_holding_days': 30,
#    'base_buffer': 0.15,
#    'concentration_limit': 0.4,
}
metrics = ['perf', 'avg_entropy']
ind = ['concentration_limit']
col = ['base_buffer']
display_heatmap(metrics, ind, col, filtering)